# WatsonxLLM

[WatsonxLLM](https://ibm.github.io/watson-machine-learning-sdk/fm_extensions.html) is wrapper for IBM [watsonx.ai](https://www.ibm.com/products/watsonx-ai) foundation models.
This example shows how to communicate with watsonx.ai models using LangChain.

Install the package [`ibm_watson_machine_learning`](https://ibm.github.io/watson-machine-learning-sdk/install.html).

In [1]:
%pip install ibm_watson_machine_learning


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
import os

os.environ["WATSONX_APIKEY"] = os.getenv("IAM_API_KEY")

## Load the model
You might need to adjust model `parameters` for different models or tasks, to do so please refer to [documentation](https://ibm.github.io/watson-machine-learning-sdk/model.html#metanames.GenTextParamsMetaNames).

In [4]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1,
}

Initialize the `WatsonxLLM` class with previous set params.

In [5]:
from langchain.llms import WatsonxLLM

watsonx_llm = WatsonxLLM(
    model_id="google/flan-ul2",
    url="https://us-south.ml.cloud.ibm.com",
    project_id=os.getenv("PROJECT_ID"),
    params=parameters,
)

Alternatively you can use Cloud Pak for Data credentials. For details, see [documentation](https://ibm.github.io/watson-machine-learning-sdk/setup_cpd.html).
```
watsonx_llm = WatsonxLLM(
    model_id='google/flan-ul2',
    url="***",
    username="***",
    password="***",
    instance_id="openshift",
    version="4.8",
    project_id='***',
    params=parameters
)
```        

## Create Chain
Create `PromptTemplate` objects which will be responsible for creating a random question.

In [6]:
from langchain.prompts import PromptTemplate

template = "Generate a random question about {topic}: Question: "
prompt = PromptTemplate.from_template(template)

Provide a topic and run the `LLMChain`.

In [7]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=watsonx_llm)
llm_chain.run("dog")

'What is the term for a dog with a long tail?'

## Calling the Model Directly
To obtain completions, you can can the model directly using string prompt.

In [8]:
# Calling a single prompt

watsonx_llm("Who is man's best friend?")

'dog'

In [9]:
# Calling multiple prompts

watsonx_llm.generate(
    [
        "The fastest dog in the world?",
        "Describe your chosen dog breed",
    ]
)

LLMResult(generations=[[Generation(text='lassie', generation_info={'finish_reason': 'eos_token'})], [Generation(text='The Basenji is a breed of dog from South Africa.', generation_info={'finish_reason': 'eos_token'})]], llm_output={'token_usage': {'generated_token_count': 18, 'input_token_count': 15}, 'model_id': 'google/flan-ul2'}, run=[RunInfo(run_id=UUID('1f038d01-a158-42ca-a7ef-90bbcaf092bc')), RunInfo(run_id=UUID('6dc7f8b3-8c8e-42e7-a0e9-dc892f916b44'))])

## Streaming the Model output 

You can stream the model output.

In [10]:
for chunk in watsonx_llm.stream(
    "Describe your favorite breed of dog and why it is your favorite."
):
    print(chunk, end="")

The Doberman pinscher is a great dog.